In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.feature_selection import SelectKBest, f_classif
import joblib
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from scipy.sparse import coo_matrix
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('../data/processed.csv', encoding='utf-8')
required_cols = ['yea_count', 'nay_count', 'nominate_mid_1', 'nominate_mid_2', 
                 'cumulative_rolls', 'action_count', 'sentiment', 'yea_dem', 'nay_rep', 'date']
for col in required_cols:
    if col not in df.columns:
        print(f'Warning: {col} missing; using default 0')
        df[col] = 0 if col != 'date' else pd.to_datetime('2023-01-01')

df['date'] = pd.to_datetime(df['date'])

# Feature Engineering
print("Creating enhanced features...")

# 1. Vote margin and ratios
df['vote_margin'] = df['yea_count'] - df['nay_count']
df['vote_ratio'] = df['yea_count'] / (df['yea_count'] + df['nay_count'] + 1e-6)
df['total_votes'] = df['yea_count'] + df['nay_count']

# 2. Party alignment features
df['party_unity_dem'] = df['yea_dem'] / (df['yea_dem'] + df['nay_rep'] + 1e-6)
df['bipartisan_score'] = 1 - abs(df['yea_dem'] - df['nay_rep']) / (df['total_votes'] + 1e-6)

# 3. Temporal features
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['year'] = df['date'].dt.year
df['day_of_week'] = df['date'].dt.dayofweek
df['is_election_year'] = (df['year'] % 4 == 0).astype(int)

# 4. Interaction features
df['sentiment_x_party'] = df['sentiment'] * df['party_unity_dem']
df['action_x_rolls'] = df['action_count'] * df['cumulative_rolls']
df['nominate_diff'] = abs(df['nominate_mid_1'] - df['nominate_mid_2'])

# 5. Statistical features
df['vote_std'] = df[['yea_count', 'nay_count']].std(axis=1)
df['vote_skewness'] = (df['yea_count'] - df['nay_count']) / (df['vote_std'] + 1e-6)

# Enhanced feature list
feature_list = [
    'yea_count', 'nay_count', 'nominate_mid_1', 'nominate_mid_2', 
    'cumulative_rolls', 'action_count', 'sentiment', 'yea_dem', 'nay_rep',
    'vote_margin', 'vote_ratio', 'total_votes', 'party_unity_dem',
    'bipartisan_score', 'month', 'quarter', 'is_election_year',
    'sentiment_x_party', 'action_x_rolls', 'nominate_diff',
    'vote_std', 'vote_skewness'
]

# Remove any NaN or infinite values
df[feature_list] = df[feature_list].replace([np.inf, -np.inf], np.nan)
df[feature_list] = df[feature_list].fillna(0)

# IMPROVED MODEL 1: Enhanced Logistic Regression with Feature Selection
print("\n1. Training Enhanced Logistic Regression...")

X = df[feature_list]
y = df['passed']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Feature selection
selector = SelectKBest(f_classif, k=15)
X_selected = selector.fit_transform(X_scaled, y)
selected_features = [feature_list[i] for i in selector.get_support(indices=True)]
print(f"Selected features: {selected_features}")

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y
)

# Grid search for best parameters
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

model_lr = GridSearchCV(
    LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    param_grid, cv=5, scoring='roc_auc', n_jobs=-1
)
model_lr.fit(X_train, y_train)

# Evaluate
preds_lr = model_lr.predict(X_test)
preds_lr_proba = model_lr.predict_proba(X_test)[:, 1]
print(f'Best params: {model_lr.best_params_}')
print(f'Logistic Regression Accuracy: {accuracy_score(y_test, preds_lr):.4f}')
print(f'ROC-AUC Score: {roc_auc_score(y_test, preds_lr_proba):.4f}')

# Save enhanced model
joblib.dump((model_lr, scaler, selector), '../models/enhanced_logistic_model.pkl')

# ENSEMBLE MODEL: Random Forest + Gradient Boosting
print("\n2. Training Ensemble Models...")

# Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_proba = rf_model.predict_proba(X_test)[:, 1]

print(f'Random Forest Accuracy: {accuracy_score(y_test, rf_preds):.4f}')
print(f'Random Forest ROC-AUC: {roc_auc_score(y_test, rf_proba):.4f}')

# Feature importance
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)
print("\nTop 10 Important Features:")
print(feature_importance.head(10))

# Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    random_state=42
)
gb_model.fit(X_train, y_train)
gb_preds = gb_model.predict(X_test)
gb_proba = gb_model.predict_proba(X_test)[:, 1]

print(f'\nGradient Boosting Accuracy: {accuracy_score(y_test, gb_preds):.4f}')
print(f'Gradient Boosting ROC-AUC: {roc_auc_score(y_test, gb_proba):.4f}')

# Ensemble prediction (weighted average)
ensemble_proba = 0.4 * rf_proba + 0.6 * gb_proba
ensemble_preds = (ensemble_proba > 0.5).astype(int)
print(f'\nEnsemble Accuracy: {accuracy_score(y_test, ensemble_preds):.4f}')
print(f'Ensemble ROC-AUC: {roc_auc_score(y_test, ensemble_proba):.4f}')

joblib.dump((rf_model, gb_model), '../models/ensemble_models.pkl')

# IMPROVED TIME SERIES: SARIMA with better preprocessing
print("\n3. Training Enhanced Time Series Model...")

# Aggregate by date with proper handling
ts_data = df.groupby('date').agg({
    'cumulative_rolls': 'mean',
    'passed': 'mean',  # Success rate
    'total_votes': 'sum'
}).sort_index()

# Fill missing dates
date_range = pd.date_range(ts_data.index.min(), ts_data.index.max(), freq='D')
ts_data = ts_data.reindex(date_range, method='ffill')

# Check stationarity
adf_test = adfuller(ts_data['cumulative_rolls'].dropna())
print(f'\nADF Statistic: {adf_test[0]:.4f}, p-value: {adf_test[1]:.4f}')

# Fit SARIMA model (with seasonal component)
try:
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    model_sarima = SARIMAX(
        ts_data['cumulative_rolls'], 
        order=(2, 1, 1),  # ARIMA parameters
        seasonal_order=(1, 0, 1, 7),  # Weekly seasonality
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    model_fit = model_sarima.fit(disp=False)
    forecast = model_fit.forecast(steps=7)
    print(f'SARIMA 7-day Forecast: {forecast.values}')
    joblib.dump(model_fit, '../models/sarima_model.pkl')
except:
    print("SARIMA failed, using standard ARIMA")
    model_arima = ARIMA(ts_data['cumulative_rolls'], order=(2,1,1))
    model_fit = model_arima.fit()
    forecast = model_fit.forecast(steps=7)
    print(f'ARIMA 7-day Forecast: {forecast.values}')
    joblib.dump(model_fit, '../models/arima_model.pkl')

# IMPROVED GNN: Better architecture and training
print("\n4. Training Enhanced Graph Neural Network...")

class ImprovedGCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout=0.5):
        super(ImprovedGCNLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, X, A):
        # Add self-loops
        A_hat = A + torch.eye(A.size(0))
        # Degree normalization
        D = torch.diag(torch.sum(A_hat, dim=1).pow(-0.5))
        A_norm = torch.matmul(torch.matmul(D, A_hat), D)
        
        # Graph convolution
        out = torch.matmul(A_norm, X)
        out = self.linear(out)
        out = F.relu(out)
        out = self.dropout(out)
        return out

class ImprovedGNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout=0.5):
        super(ImprovedGNN, self).__init__()
        self.gcn1 = ImprovedGCNLayer(input_dim, hidden_dims[0], dropout)
        self.gcn2 = ImprovedGCNLayer(hidden_dims[0], hidden_dims[1], dropout)
        self.gcn3 = ImprovedGCNLayer(hidden_dims[1], hidden_dims[2], dropout)
        
        # Graph-level prediction layers
        # Now operating on node features directly
        self.fc1 = nn.Linear(hidden_dims[2], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, X, A):
        # Graph convolutions
        x = self.gcn1(X, A)
        x = self.gcn2(x, A)
        x = self.gcn3(x, A)
        
        # Node-level predictions (no pooling)
        # Apply FC layers to each node
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

# Create a more realistic graph structure
# In practice, this should be based on actual sponsor/cosponsor relationships
num_nodes = min(100, len(df))
G = nx.watts_strogatz_graph(num_nodes, 20, 0.3)  # Small-world network
adj = nx.adjacency_matrix(G).todense()
A = torch.tensor(adj, dtype=torch.float)

# Prepare node features (use selected features)
X_gnn = torch.tensor(X_selected[:num_nodes], dtype=torch.float)
y_gnn = torch.tensor(y.values[:num_nodes], dtype=torch.float).view(-1, 1)

# Split data
train_size = int(0.8 * num_nodes)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:train_size] = True
val_mask = ~train_mask

# Initialize model with better architecture
model_gnn = ImprovedGNN(
    input_dim=X_gnn.shape[1],
    hidden_dims=[64, 32, 16],
    output_dim=1,
    dropout=0.3
)

# Training with better optimization
optimizer = torch.optim.Adam(model_gnn.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)
criterion = nn.BCELoss()

best_val_loss = float('inf')
patience_counter = 0

for epoch in range(300):
    # Training
    model_gnn.train()
    optimizer.zero_grad()
    pred = model_gnn(X_gnn, A)
    
    # Only compute loss on training nodes
    train_loss = criterion(pred[train_mask], y_gnn[train_mask])
    train_loss.backward()
    optimizer.step()
    
    # Validation
    model_gnn.eval()
    with torch.no_grad():
        val_pred = model_gnn(X_gnn, A)
        val_loss = criterion(val_pred[val_mask], y_gnn[val_mask])
    
    scheduler.step(val_loss)
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model_gnn.state_dict(), '../models/best_gnn_model.pth')
    else:
        patience_counter += 1
        if patience_counter > 20:
            print(f"Early stopping at epoch {epoch}")
            break
    
    if epoch % 20 == 0:
        train_acc = ((pred[train_mask] > 0.5) == y_gnn[train_mask]).float().mean()
        val_acc = ((val_pred[val_mask] > 0.5) == y_gnn[val_mask]).float().mean()
        print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
              f'Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

# Load best model
model_gnn.load_state_dict(torch.load('../models/best_gnn_model.pth'))
print("\nGNN training completed!")

Creating enhanced features...

1. Training Enhanced Logistic Regression...
Selected features: ['yea_count', 'nominate_mid_1', 'cumulative_rolls', 'nay_rep', 'vote_margin', 'vote_ratio', 'total_votes', 'party_unity_dem', 'bipartisan_score', 'month', 'quarter', 'is_election_year', 'sentiment_x_party', 'vote_std', 'vote_skewness']
Best params: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Logistic Regression Accuracy: 0.9241
ROC-AUC Score: 0.9597

2. Training Ensemble Models...
Random Forest Accuracy: 0.9385
Random Forest ROC-AUC: 0.9848

Top 10 Important Features:
              feature  importance
0           yea_count    0.273462
6         total_votes    0.158436
4         vote_margin    0.119240
14      vote_skewness    0.098621
3             nay_rep    0.080893
5          vote_ratio    0.077043
8    bipartisan_score    0.034291
12  sentiment_x_party    0.034286
13           vote_std    0.033067
1      nominate_mid_1    0.029906

Gradient Boosting Accuracy: 0.9548
Gradient Boosting 